# Object Classification

### **Exteracting Zip Files**

In [0]:
from zipfile import ZipFile
fname="cifar10classwiseimages.zip"
with ZipFile(fname,'r') as zip:
  zip.extractall()
  print("done")

done


### **Import Open-cv Library**

In [0]:
import cv2
from google.colab.patches import cv2_imshow
from glob import glob
import numpy as np

## Training Data

In [0]:
import cv2
from google.colab.patches import cv2_imshow
from glob import glob
import numpy as np
img_pixels_train=np.zeros((45000,32,32))
labels_train=np.zeros((45000,))
images=['0','1','2','3','4','5','6','7','8']
i=0
for j in range(len(images)):
  
  print(images[j])
  data_db=images[j]
  img_dbb=glob('cifar/train/'+data_db+'/*.png')
  
  for k in img_dbb:
    img=cv2.imread(k,1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      
    img_pixels_train[i]=gray
    labels_train[i]=j
    i+=1

print(i)

0
1
2
3
4
5
6
7
8
45000


**Shape of train samples and Train labels**

In [0]:
print((img_pixels_train.shape))
#abels=labels.reshape(920,1)
print(labels_train.shape)

(45000, 32, 32)
(45000,)


## Testing Data

In [0]:
import cv2
from google.colab.patches import cv2_imshow
from glob import glob
import numpy as np
img_pixels_test=np.zeros((9000,32,32))
labels_test=np.zeros((9000,))
images=['0','1','2','3','4','5','6','7','8']
i=0
for j in range(len(images)):
  
  print(images[j])
  data_db=images[j]
  img_dbb=glob('cifar/test/'+data_db+'/*.png')
  
  for k in img_dbb:
    img=cv2.imread(k,1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      
    img_pixels_test[i]=gray
    labels_test[i]=j
    i+=1

print(i)

0
1
2
3
4
5
6
7
8
9000


**Shape of test sample and test labels**

In [0]:
print((img_pixels_test.shape))
#abels=labels.reshape(920,1)
print(labels_test.shape)

(9000, 32, 32)
(9000,)


**Convert data type as float**

In [0]:
def convert_dtype(x):
    
    x_float = x.astype('float32')
    return x_float

train_samples = convert_dtype(img_pixels_train)
test_samples = convert_dtype(img_pixels_test)
print(train_samples.shape)
print(test_samples.shape)

(45000, 32, 32)
(9000, 32, 32)


**Normalizing the data between 0-1**

In [0]:
def normalize(x):
    
    x_n=x/255
    return x_n
    
train_samples = normalize(train_samples)
test_samples = normalize(test_samples)
print(train_samples.shape)
print(test_samples.shape)

(45000, 32, 32)
(9000, 32, 32)


**One hot encoding of train and test labels**

In [0]:
def oneHot(y, Ny):
    
    from keras.utils import to_categorical
    y_oh=to_categorical(y,Ny)
    return y_oh


train_labels = oneHot(labels_train, 9)
test_labels = oneHot(labels_test, 9)
print(train_labels[39000])
print(test_labels.shape)

[0. 0. 0. 0. 0. 0. 0. 1. 0.]
(9000, 9)


**Reshape the data**

In [0]:
def reshape(x):
    
    print(x.shape)
    x_r=x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
    return x_r

train_samples = reshape(train_samples)
print(train_samples.shape)

test_samples = reshape(test_samples)
print(test_samples.shape)

(45000, 32, 32)
(45000, 32, 32, 1)
(9000, 32, 32)
(9000, 32, 32, 1)


# **Keras model using CNN**

In [0]:
import keras
from keras.layers import Dense,Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation
from keras.layers import Dropout
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Dropout

def create_model():
    
    model=Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,1), kernel_regularizer=l2(0.01)))
    model.add(Conv2D(32, (3, 3), padding='same',activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    model.add(Dropout(0.2))    
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))  
    
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(9, activation='softmax'))
    
    
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

model = create_model()

## Model Fitting

In [0]:
history = model.fit(train_samples, train_labels, validation_split = 0.1, epochs=2, batch_size=30)

Train on 40500 samples, validate on 4500 samples
Epoch 1/2
40500/40500 [==============================] - 386s 10ms/step - loss: 0.2394 - acc: 0.9123 - val_loss: 0.5064 - val_acc: 0.8519
Epoch 2/2
40500/40500 [==============================] - 385s 10ms/step - loss: 0.1705 - acc: 0.9349 - val_loss: 0.4454 - val_acc: 0.8310


### Model summary

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
batch_normalization_31 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
batch_normalization_32 (Batc (None, 15, 15, 64)        256       
__________

### **Predicted value**

In [0]:
y=model.predict(test_samples)
print(y.shape)

(9000, 9)


### **Confusion metrics using Sklearn**

In [0]:
from sklearn.metrics import confusion_matrix
  
cm = confusion_matrix(test_labels.argmax(axis=1), y.argmax(axis=1))
print(cm)
loss,acc=model.evaluate(test_samples, test_labels,verbose=0)
print(loss,acc)
           

[[892  39  34  19   4   0   2   8   2]
 [ 31 947   6   6   2   1   3   3   1]
 [196  26 546 119  12  23  31  47   0]
 [135  52  97 535  18  33  55  75   0]
 [175  45 156 156 247  12  61 148   0]
 [ 85  39 140 291  13 301  21 110   0]
 [ 53  52  78 134   4   9 655  15   0]
 [ 65  44  58  56  10   5   9 753   0]
 [461 333  26  64   0   3  11  10  92]]
0.22115873788462745 0.9173086366653442


# Testing the model

**Testing : 1**

In [0]:
p=cv2.imread('photo-1529074963764-98f45c47344b.jpg')
cv2.imshow(p)

gray = cv2.cvtColor(p, cv2.COLOR_BGR2GRAY)
gray=cv2.resize(gray,(32,32))
gray=gray/255

sample = convert_dtype(gray)

sample=sample.reshape((1,32,32))

sample = reshape(sample)

yoyo=model.predict(sample)

(1, 32, 32)
(1, 32, 32)
(1, 32, 32, 1)
[0]


**Result :**

In [0]:
res=yoyo.argmax(axis=1)
print("Class of image is : ",images[yoyo])

**Testing : 2**

In [0]:
p=cv2.imread('aHR0cDovL3d3dy5saXZlc2NpZW5jZS5jb20vaW1hZ2VzL2kvMDAwLzA5Ni8yODEvb3JpZ2luYWwvd2hpdGUtdHJlZS1mcm9nLmpwZw==.webp.png')
cv2.imshow(p)

gray = cv2.cvtColor(p, cv2.COLOR_BGR2GRAY)
gray=cv2.resize(gray,(32,32))
gray=gray/255

sample = convert_dtype(gray)

sample=sample.reshape((1,32,32))

sample = reshape(sample)

yoyo=model.predict(sample)


(1, 32, 32)
(1, 32, 32)
(1, 32, 32, 1)
[6]


**Result :**

In [0]:
res=yoyo.argmax(axis=1)
print("Class of image is : ",images[yoyo])